<a href="https://colab.research.google.com/github/kcw0331/Deeplearning/blob/main/text_generation2021%EB%85%845%EC%9B%9426%EC%9D%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Text generation with an RNN

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/text/tutorials/text_generation"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/text/blob/master/docs/tutorials/text_generation.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/text/blob/master/docs/tutorials/text_generation.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/text/docs/tutorials/text_generation.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This tutorial demonstrates how to generate text using a character-based RNN. You will work with a dataset of Shakespeare's writing from Andrej Karpathy's [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/). Given a sequence of characters from this data ("Shakespear"), train a model to predict the next character in the sequence ("e"). Longer sequences of text can be generated by calling the model repeatedly.

Note: Enable GPU acceleration to execute this notebook faster. In Colab: *Runtime > Change runtime type > Hardware accelerator > GPU*.

This tutorial includes runnable code implemented using [tf.keras](https://www.tensorflow.org/guide/keras/sequential_model) and [eager execution](https://www.tensorflow.org/guide/eager). The following is the sample output when the model in this tutorial trained for 30 epochs, and started with the prompt "Q":

<pre>
QUEENE:
I had thought thou hadst a Roman; for the oracle,
Thus by All bids the man against the word,
Which are so weak of care, by old care done;
Your children were in your holy love,
And the precipitation through the bleeding throne.

BISHOP OF ELY:
Marry, and will, my lord, to weep in such a one were prettiest;
Yet now I was adopted heir
Of the world's lamentable day,
To watch the next way with his father with his face?

ESCALUS:
The cause why then we are all resolved more sons.

VOLUMNIA:
O, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, it is no sin it should be dead,
And love and pale as any will to that word.

QUEEN ELIZABETH:
But how long have I heard the soul for this world,
And show his hands of life be proved to stand.

PETRUCHIO:
I say he look'd on, if I must be content
To stay him from the fatal of our country's bliss.
His lordship pluck'd from this sentence then for prey,
And then let us twain, being the moon,
were she such a case as fills m
</pre>

While some of the sentences are grammatical, most do not make sense. The model has not learned the meaning of words, but consider:

* The model is character-based. When training started, the model did not know how to spell an English word, or that words were even a unit of text.

* The structure of the output resembles a play—blocks of text generally begin with a speaker name, in all capital letters similar to the dataset.

* As demonstrated below, the model is trained on small batches of text (100 characters each), and is still able to generate a longer sequence of text with coherent structure.

## Setup

- 여기에서는 셰익스피어의 소설을 데이터로 사용을 한다.

### Import TensorFlow and other libraries

- TensorFlow를 불러와 준다.

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

import numpy as np
import os
import time

### Download the Shakespeare dataset

Change the following line to run this code on your own data.

- 셰익스 피어의 데이터셋을 불러와준다.

In [ ]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1122304/1115394 [==============================] - 0s 0us/step


### Read the data

First, look in the text:

In [ ]:
path_to_file  

'/root/.keras/datasets/shakespeare.txt'

- path_to_file을 쳐서 보았을때, /root/.keras/datasets/shakespeare.txt 여기에 다운 받아져 있는 것을 볼 수 있다.

In [ ]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8') #utf-8이 형식으로 해서 출력해봐라 하는 것이다.
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 1115394 characters


- 전체는 1115394 characters로 구성이 되어있는 것을 볼 수 있다.

In [ ]:
# Take a look at the first 250 characters in text
print(text[:250])   #text를 앞에서부터 250까지 출력을 해주는 것이다.

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [ ]:
# The unique characters in the file
vocab = sorted(set(text))  
print(f'{len(vocab)} unique characters')

65 unique characters


- text에 단어들이 몇개사용이 되었는지 index가 필요한데, 그래서 
vocab해서 65 unique characters이 들어 있는 것을 알 수 있다.

## Process the text

### Vectorize the text

Before training, you need to convert the strings to a numerical representation. 

The `preprocessing.StringLookup` layer can convert each character into a numeric ID. It just needs the text to be split into tokens first.

- text를 수치형으로 Vectorize해주는 방법이다.

In [ ]:
idx2char

array(['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?',
       'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
       'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
       'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm',
       'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z'],
      dtype='<U1')

In [ ]:
char2idx

{'\n': 0,
 ' ': 1,
 '!': 2,
 '$': 3,
 '&': 4,
 "'": 5,
 ',': 6,
 '-': 7,
 '.': 8,
 '3': 9,
 ':': 10,
 ';': 11,
 '?': 12,
 'A': 13,
 'B': 14,
 'C': 15,
 'D': 16,
 'E': 17,
 'F': 18,
 'G': 19,
 'H': 20,
 'I': 21,
 'J': 22,
 'K': 23,
 'L': 24,
 'M': 25,
 'N': 26,
 'O': 27,
 'P': 28,
 'Q': 29,
 'R': 30,
 'S': 31,
 'T': 32,
 'U': 33,
 'V': 34,
 'W': 35,
 'X': 36,
 'Y': 37,
 'Z': 38,
 'a': 39,
 'b': 40,
 'c': 41,
 'd': 42,
 'e': 43,
 'f': 44,
 'g': 45,
 'h': 46,
 'i': 47,
 'j': 48,
 'k': 49,
 'l': 50,
 'm': 51,
 'n': 52,
 'o': 53,
 'p': 54,
 'q': 55,
 'r': 56,
 's': 57,
 't': 58,
 'u': 59,
 'v': 60,
 'w': 61,
 'x': 62,
 'y': 63,
 'z': 64}

In [ ]:
#Creating a mapping from unique characters to indices
char2idx = {u : i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [ ]:
text_as_int[:100]  #chatacter를 수치형으로 나타낸 것이다.

array([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43,
       44, 53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39,
       52, 63,  1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1,
       51, 43,  1, 57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31,
       54, 43, 39, 49,  6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56,
       57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 37, 53, 59])

Now we have an integer representation for each character. Notice that we mapped the character as indexs from 0 to len(unique).

In [ ]:
print('{')
for char,_ in zip(char2idx, range(20)):
  print(' {:4s}: {:3d}, '.format(repr(char), char2idx[char]))

{
 '\n':   0, 
 ' ' :   1, 
 '!' :   2, 
 '$' :   3, 
 '&' :   4, 
 "'" :   5, 
 ',' :   6, 
 '-' :   7, 
 '.' :   8, 
 '3' :   9, 
 ':' :  10, 
 ';' :  11, 
 '?' :  12, 
 'A' :  13, 
 'B' :  14, 
 'C' :  15, 
 'D' :  16, 
 'E' :  17, 
 'F' :  18, 
 'G' :  19, 


In [ ]:
#Show how the first 13 characters from the text are mapped to integers #이거는 캐릭터들을 출력해준 것이다.
print('{} --- charaters mapped to int ---> {}'.format(repr(text[:13]), text_as_int[:13])) 

'First Citizen' --- charaters mapped to int ---> [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [ ]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

Now create the `preprocessing.StringLookup` layer:

In [ ]:
ids_from_chars = preprocessing.StringLookup(
    vocabulary=list(vocab), mask_token=None)

It converts form tokens to character IDs:

In [ ]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

Since the goal of this tutorial is to generate text, it will also be important to invert this representation and recover human-readable strings from it. For this you can use `preprocessing.StringLookup(..., invert=True)`.  

Note: Here instead of passing the original vocabulary generated with `sorted(set(text))` use the `get_vocabulary()` method of the `preprocessing.StringLookup` layer so that the `[UNK]` tokens is set the same way.

In [ ]:
chars_from_ids = tf.keras.layers.experimental.preprocessing.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

This layer recovers the characters from the vectors of IDs, and returns them as a `tf.RaggedTensor` of characters:

In [ ]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

You can `tf.strings.reduce_join` to join the characters back into strings. 

In [ ]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [ ]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

### The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task you're training the model to perform. The input to the model will be a sequence of characters, and you train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?


### Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain `seq_length` characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of `seq_length+1`. For example, say `seq_length` is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the `tf.data.Dataset.from_tensor_slices` function to convert the text vector into a stream of character indices.

In [ ]:
#The maximum legth sentance we want for a single input in charaters
seq_length = 100
examples_per_epoch = len(text) // (seq_length+1)

#Create training examples / rargets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i .numpy()])

F
i
r
s
t


- tf.data.Dataset.from_tensor_slices이 함수를 통해서 데이터 정리를 더 쉽게 할 수 있다. 

The `batch` method lets you easily convert these individual characters to sequences of the desired size.

In [ ]:
seqences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in seqences.take(5):  #seqences를 받아와서 5개를 출력해주는 것이다.
  print(repr(''.join(idx2char[item.numpy()])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


- batch만큼 seqences length가 딸려 오는 것을 볼 수 있다.
- 위에 출력된 얘들이 학습을 하는데 사용을 한다.

For each seqence, duplicate and shift it to form the input and target text by using the map method to apply a simple function to each batch:

In [ ]:
def split_input_target(chunk):
  input_text = chunk[:-1]   
  target_text = chunk[1:]
  return input_text, target_text

dataset = seqences.map(split_input_target)

Print the first examples input and target values:

In [ ]:
for input_example, target_example in dataset.take(1):
  print('input data', repr(''.join(idx2char[input_example.numpy()])))
  print('target data', repr(''.join(idx2char[target_example.numpy()])))

input data 'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
target data 'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


Each index of these vectors are processed as on time step. For the input at time step 0, the model recives the index for "F" and trys to predict the index for "i" as the next character. At the next timestep, it does the same thing but the RNN considers the previous step context in addition to the current input character. 

In [ ]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
  print("Step {:40}".format(i))
  print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
  print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step                                        0
  input: 18 ('F')
  expected output: 47 ('i')
Step                                        1
  input: 47 ('i')
  expected output: 56 ('r')
Step                                        2
  input: 56 ('r')
  expected output: 57 ('s')
Step                                        3
  input: 57 ('s')
  expected output: 58 ('t')
Step                                        4
  input: 58 ('t')
  expected output: 1 (' ')


- input과 ouput에 대한 example이며 input이 이렇게 나오면 ouput은 이렇게 나와야 한다는 것이다.

### Create training batches

You used `tf.data` to split the text into manageable sequences. But before feeding this data into the model, you need to shuffle the data and pack it into batches.

In [ ]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)  #이런식으로 dataset을 만들 수 있다.

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

10000개를 64개 해서 트레인 해준다.

## Build The Model

This section defines the model as a `keras.Model` subclass (For details see [Making new Layers and Models via subclassing](https://www.tensorflow.org/guide/keras/custom_layers_and_models)). 

This model has three layers:

* `tf.keras.layers.Embedding`: The input layer. A trainable lookup table that will map each character-ID to a vector with `embedding_dim` dimensions;
* `tf.keras.layers.GRU`: A type of RNN with size `units=rnn_units` (You can also use an LSTM layer here.)
* `tf.keras.layers.Dense`: The output layer, with `vocab_size` outputs. It outputs one logit for each character in the vocabulary. These are the log-likelihood of each character according to the model.

In [ ]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model =tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,  #맨처음 데이터에 embedding layer를 달아가지고 
                             batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                       return_sequences=True,
                       stateful=True,
                       recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

- 우리가 책에서 배운 것과 다른점은 여기에서는 embedding layer를 달아 주었다.

In [ ]:
model = build_model(
    vocab_size = len(vocab),
    embedding_dim = embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

For each character the model looks up the embedding, runs the GRU one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-likelihood of the next character:

A drawing of the data paasing through the model 

Please note that we choose to Keras sequential model here since all the layers in the model only have single input and produce single output. In case you want to retrieve and reuse the states from stateful RNN layer, you might want to build your model with Keras functional API or model subclassing. Please ckeck Keras RNN guide for more details. 

## Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [ ]:
input_example_batch  #이게 바로 수치형으로 바뀌니깐 이걸바로 모델에 집어넣어서 해준다.

<tf.Tensor: shape=(64, 100), dtype=int64, numpy=
array([[56,  1, 47, ..., 58, 39, 45],
       [ 0, 23, 21, ...,  0,  0, 31],
       [35, 46, 47, ..., 61,  1, 61],
       ...,
       [ 1, 52, 53, ...,  1, 57, 49],
       [58, 53,  1, ...,  0,  0, 26],
       [56,  1, 50, ..., 47, 58, 46]])>

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


- dataset으로 부터 하나를 떼와서 input과 target을 따와서 example_batch_predictions을 해보는 거다.

In the above example the sequence length of the input is `100` but the model can be run on inputs of any length:

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


To get actual predictions from the model you need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

Note: It is important to _sample_ from this distribution as taking the _argmax_ of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

- 모델을 학습 시킨다음에 캐릭터들을 제너레이터 하겠다는 것이다.

In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)

In [ ]:
sampled_indices

<tf.Tensor: shape=(100, 1), dtype=int64, numpy=
array([[25],
       [38],
       [19],
       [ 6],
       [60],
       [48],
       [ 6],
       [42],
       [55],
       [39],
       [47],
       [54],
       [64],
       [52],
       [31],
       [41],
       [46],
       [15],
       [50],
       [ 2],
       [33],
       [64],
       [41],
       [25],
       [ 0],
       [41],
       [56],
       [37],
       [31],
       [45],
       [60],
       [28],
       [ 6],
       [58],
       [47],
       [59],
       [10],
       [26],
       [45],
       [60],
       [30],
       [26],
       [54],
       [12],
       [46],
       [35],
       [10],
       [37],
       [17],
       [16],
       [55],
       [27],
       [18],
       [59],
       [56],
       [ 0],
       [44],
       [51],
       [32],
       [33],
       [10],
       [40],
       [ 6],
       [ 1],
       [64],
       [45],
       [14],
       [61],
       [ 7],
       [ 3],
       [51],
       [30],
       [43],
   

In [ ]:
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [ ]:
sampled_indices #한 차원 빼준거를 넘파이로 만들어 주었다.

array([25, 38, 19,  6, 60, 48,  6, 42, 55, 39, 47, 54, 64, 52, 31, 41, 46,
       15, 50,  2, 33, 64, 41, 25,  0, 41, 56, 37, 31, 45, 60, 28,  6, 58,
       47, 59, 10, 26, 45, 60, 30, 26, 54, 12, 46, 35, 10, 37, 17, 16, 55,
       27, 18, 59, 56,  0, 44, 51, 32, 33, 10, 40,  6,  1, 64, 45, 14, 61,
        7,  3, 51, 30, 43, 27, 43, 43, 22,  0, 37, 12,  4, 20, 16,  3, 24,
        1, 15,  9, 35,  2, 63, 31, 55, 37, 36, 23, 10, 36, 49, 20])

In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

This gives us, at each timestep, a prediction of the next character index:

In [ ]:
sampled_indices

array([16, 24, 10, 61,  4, 21,  4, 64, 30, 27, 56, 57, 16,  6, 55,  9, 52,
       30, 55, 51, 48, 32, 28, 38, 29,  4,  9, 58, 14, 62, 58, 25, 50, 22,
       38, 43, 59, 28, 37, 18, 32, 32, 30, 24, 51, 42, 24, 13, 14, 43, 36,
        9, 39, 28, 13, 11, 30, 19, 25, 13, 53, 51, 61, 16,  4, 36, 62, 21,
       29, 44, 23, 37, 46, 44, 34,  1, 43,  1, 49, 21, 47, 42, 26, 64, 58,
       16, 62,  4, 18, 11, 44,  9, 19, 27, 17, 45, 17,  5,  6, 23])

Decode these to see the text predicted by this untrained model:

In [ ]:
print("input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices])))

input: 
 'r is come.\n\nFRIAR PETER:\nCome, I have found you out a stand most fit,\nWhere you may have such vantag'

Next Char Predictions: 
 "DL:w&I&zROrsD,q3nRqmjTPZQ&3tBxtMlJZeuPYFTTRLmdLABeX3aPA;RGMAomwD&XxIQfKYhfV e kIidNztDx&F;f3GOEgE',K"


- idx2char를 사용해서 index를 charater로 바꾸어 주는 작업을 해준다.

## Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

### Attach an optimizer, and a loss function

The standard `tf.keras.losses.sparse_categorical_crossentropy` loss function works in this case because it is applied across the last dimension of the predictions.

Because your model returns logits, you need to set the `from_logits` flag.


In [ ]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions) #괄호안에 있는거를 비교해서 loss를 계산해준다.
print("Predition shape: ", example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")
print("scalar_loss:     ", example_batch_loss.numpy().mean()) #이번에는 결과가 multiple이여서 평균을 내서 정하는 것 같다.

Predition shape:  (64, 100, 65) # (batch_size, sequence_length, vocab_size)
scalar_loss:      4.1734223


Configure the training procedure using the `tf.keras.Model.compile` method. Use `tf.keras.optimizers.Adam` with default arguments and the loss function.

In [ ]:
model.compile(optimizer='adam', loss=loss)  #이 loss를 기준으로 모델을 컴파일 해준다.

### Configure checkpoints

Use a `tf.keras.callbacks.ModelCheckpoint` to ensure that checkpoints are saved during training:

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

- 모델은 말고 weights만 저장되게 해 놨다.

### Execute the training

To keep training time reasonable, use 10 epochs to train the model. In Colab, set the runtime to GPU for faster training.

In [ ]:
EPOCHS = 10

In [ ]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
172/172 [==============================] - 9s 40ms/step - loss: 2.6478
Epoch 2/10
172/172 [==============================] - 8s 40ms/step - loss: 1.9498
Epoch 3/10
172/172 [==============================] - 8s 40ms/step - loss: 1.6867
Epoch 4/10
172/172 [==============================] - 8s 40ms/step - loss: 1.5413
Epoch 5/10
172/172 [==============================] - 8s 40ms/step - loss: 1.4553
Epoch 6/10
172/172 [==============================] - 8s 40ms/step - loss: 1.3951
Epoch 7/10
172/172 [==============================] - 8s 40ms/step - loss: 1.3499
Epoch 8/10
172/172 [==============================] - 8s 40ms/step - loss: 1.3118
Epoch 9/10
172/172 [==============================] - 8s 40ms/step - loss: 1.2770
Epoch 10/10
172/172 [==============================] - 8s 40ms/step - loss: 1.2448


## Generate text

### Restore the latest checkpoint
To keep this prediction step simple, use a batch size of 1.
Because of the way the RNN state is passed from timestep, the model only accepts a fixed batch size once built.
To run the model with a different batch_size, we need to rebuild the model and restore the weigths from the checkpoint.

In [ ]:
tf.train.latest_checkpoint(checkpoint_dir)  #먼저 latest_checkpoint이걸 가지고 온다.

'./training_checkpoints/ckpt_10'

In [ ]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))  #latest_checkpoint이걸로 부터 weigths를 할 수 있다.

model.build(tf.TensorShape([1, None]))

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 65)             66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


## The predition loop
The following code block generates the text:
 - It Starts by choosing a start string, initializing the RNN state and setting the number of charaters to generate.
 - Get the prediction distribution of the next character using the start string and RNN state.
 - Then, use a categorical distribution to calculate the index of the predicted chatater. Use this predicted character as our next input to the model.
 - The RNN state returned by the model is fed back into the model so that it now has more context, instead than only one character. After predicting the next character, the modified RNN states are again fed back into the model, which is how it leans as it gets more context from the previously predicted characters.
To generate text the model's output is fed back to the input.

Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates a Shakespeare-like wirting vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences. 



- prediction loop를 돌텐데 이건 genertate_text부분이라고 말씀하심.

In [ ]:
def generate_text(model, start_string):
  #Evaluation step (generating text using the learned model)
  
  #Number of characters to generate 
  num_generate = 100  #generate는 1000단어를 generate할 거다

  #Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  #Empty string to store our results
  text_generated = []

  #Low temperatures results in more predictable text.
  #Higher temperatures results in more surprising text.
  #Experiment to find the best setting.
  temperature = 1.0

  #Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
    predictions = model(input_eval)
    #remove the batch dimension
    predictions = tf.squeeze(predictions, 0)

    #using a categorical distribution to predict the character returned by the model
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy() 

    #We pass the predicted character as the next input to the model
    #along with the previous hidden state
    input_eval = tf.expand_dims([predicted_id], 0)

    text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [ ]:
print(generate_text(model, start_string=u"ROMEO: "))

ROMEO: masters, pardon!

Nurse:
No worring fulf-lord, keeps I heard--about him speak to fish.

WARWICK:
We 


The easitest thing you can do to improve the results it to train it for longer(try EPOCHS=30).
You can also experiment with a different start string. or try adding another RNN layer to improve the model's accuracy, or adjustring the temperature papameter to generate more or less random preditions. 

- 여기까지 설명하시고 이 아래코드는 관심있는 학생들 한번 보라고 말씀하심.

# Advanced: Customized Training
The above training procedure is simple, but does not give you much control.

So now that you've seen how to run the model manually let's unpack

In [ ]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [ ]:
optimizer = tf.keras.optimizers.Adam()

In [ ]:
@tf.function
def train_step(inp, target):
  with tf.GradientTape() as tape:
    predictions = model(inp)
    loss = tf.reduce_mean(
        tf.keras.losses.sparse_categorical_crossentropy(
            target.predictions, from_logits=True))
  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  return loss

In [ ]:
#Training step
EPOCHS =10

for epoch in range(EPOCHS):
  start = time.time()


  #initializing the hidden state at the start of every epoch
  #initally hidden in None
  hidden = model.reset_states()

  for (batch_n, (inp, target)) in enumerate(dataset):
    loss = train_step(inp, target)

    if batch_n % 100 ==0:
      template ='Epoch {} Batch {} Loss{}'
      print(template.format(epoch+1, batch_n, loss))

  #saving (checkpoint) the model every 5 epochs
  if (epoch + 1) % 5 ==0:
    model.save_weights(checkpoint_preflx.format(epoch=epoch))
  
  print ('Epoch {} Loss{:4f}'.format(epoch+1, loss))
  print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

model.save_weights(checkpoint_preflx.format(epoch=epoch))

AttributeError: ignored